### Q1 Getting the embeddings model

In [2]:
# Import libraries
from sentence_transformers import SentenceTransformer

# Get the model
embedding_model = SentenceTransformer("multi-qa-distilbert-cos-v1")

# Define the question
user_question = "I just discovered the course. Can I still join it?"

# Create a vector
q_vector = embedding_model.encode(user_question)

print("Answer 1: ", q_vector[0])

/home/dmitry/venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-07-13 16:23:05.874201: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-13 16:23:05.891785: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-13 16:23:05.915834: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been r

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.52k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Answer 1:  0.0782226


In [6]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

### Q2 Creating the embeddings

In [21]:
# Create a list
embeddings = []

# Iterate over each document
for doc in documents:
    qa_text = f'{'question'} {'text'}'
    
    # Compute the embeddings
    qa_text_vector = embedding_model.encode(qa_text)
    embeddings.append(qa_text_vector)

# Put all embeddings into a single matrix
import numpy as np
X = np.array(embeddings)

# Check the shape
print("Answer 2:", X.shape)

Answer 2: (948, 768)


### Q3 Computing cosine similarity

In [8]:
scores = X.dot(q_vector)
print(scores)

[0.04935043 0.04935043 0.04935043 0.04935043 0.04935043 0.04935043
 0.04935043 0.04935043 0.04935043 0.04935043 0.04935043 0.04935043
 0.04935043 0.04935043 0.04935043 0.04935043 0.04935043 0.04935043
 0.04935043 0.04935043 0.04935043 0.04935043 0.04935043 0.04935043
 0.04935043 0.04935043 0.04935043 0.04935043 0.04935043 0.04935043
 0.04935043 0.04935043 0.04935043 0.04935043 0.04935043 0.04935043
 0.04935043 0.04935043 0.04935043 0.04935043 0.04935043 0.04935043
 0.04935043 0.04935043 0.04935043 0.04935043 0.04935043 0.04935043
 0.04935043 0.04935043 0.04935043 0.04935043 0.04935043 0.04935043
 0.04935043 0.04935043 0.04935043 0.04935043 0.04935043 0.04935043
 0.04935043 0.04935043 0.04935043 0.04935043 0.04935043 0.04935043
 0.04935043 0.04935043 0.04935043 0.04935043 0.04935043 0.04935043
 0.04935043 0.04935043 0.04935043 0.04935043 0.04935043 0.04935043
 0.04935043 0.04935043 0.04935043 0.04935043 0.04935043 0.04935043
 0.04935043 0.04935043 0.04935043 0.04935043 0.04935043 0.0493

### Search function

In [24]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=5):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(q_vector, num_results=5)

[{'text': 'Problem description\nPre-commit command was failing with isort repo.\nSolution description\nSet version to 5.12.0\nAdded by Erick Calderin',
  'section': 'Module 6: Best practices',
  'question': 'Isort Pre-commit',
  'course': 'mlops-zoomcamp',
  'id': 'fb3c4150'},
 {'text': 'Problem description\nIf you are having problems with the integration tests and kinesis double check that your aws regions match on the docker-compose and local config. Otherwise you will be creating a stream in the wrong region\nSolution description\nFor example set ~/.aws/config region = us-east-1 and the docker-compose.yaml - AWS_DEFAULT_REGION=us-east-1\nAdded by Quinn Avila',
  'section': 'Module 6: Best practices',
  'question': 'AWS regions need to match docker-compose',
  'course': 'mlops-zoomcamp',
  'id': '66326a87'},
 {'text': "Problem: About week_2 homework: The register_model.py  script, when I copy it into a jupyter notebook fails and spits out the following error. AttributeError: 'tuple' 

### Q4 Hit-rate for the search engine

In [11]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [22]:
# Evaluate the search function
from tqdm.auto import tqdm
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total)
    }

# Calculate the hit rate
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

evaluate(ground_truth, search_engine.search)

  0%|          | 0/1830 [00:00<?, ?it/s]

TypeError: unsupported operand type(s) for *: 'float' and 'dict'